In [ ]:
# initialize
import mne
import os
import scipy.io
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from scipy import signal
from scipy import stats
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')
from itertools import permutations,combinations
from IPython.display import clear_output
import seaborn as sns
from scipy.linalg import toeplitz
from numpy import linalg as LA
from mne.event import define_target_events
import gcmi

data_path = os.path.dirname(os.path.dirname(os.getcwd()))


save_path = data_path + '/python/data/coherence'
info = mne.io.read_raw_fif((save_path+'-info'),preload=True)

subject_name = ['Alice','Andrea','Daniel','Elena','Elenora','Elisa','Federica','Francesca','Gianluca1','Giada','Giorgia',
                'Jonluca','Laura','Leonardo','Linda','Lucrezia','Manu','Marco','Martina','Pagani','Pasquale','Sara',
                'Silvia','Silvia2','Tommaso']


save_path = data_path + '/python/data/extracted_features/features.pkl'
feat = pd.read_pickle(save_path)

clear_output()

In [ ]:
# data preprocessing parameters
remove_first = 0.5 #second
new_sampling_rate = 100

no_surrogate = 1000
features = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
features = ['envelop']
features = ['jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']


con = ['hyper','normal','hypo','All']
con = 'All'

trial_len = 5 #(greater than)second 

apply_delay = False
delay = [0]

# cca parameters


# CCA functions
def nanRXY(X,Y):
    D = X.shape[0]
    x = np.vstack([X,Y])
    RXY = np.cov(x)
    
    Rxx = RXY[0:D,0:D]
    Ryy = RXY[D:,D:]
    Rxy = RXY[0:D,D:]
    Ryx = RXY[D:,0:D]
    return Rxx,Ryy,Rxy,Ryx
    
def regInv(R,K,typeF):
    
    eigenValues,eigenVectors = LA.eigh(R)
    idx = np.argsort(eigenValues)
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    
    if(typeF=='sqrt'):
        d = 1/np.sqrt(eigenValues[-K:])  # regularized sqrt inverse
    else:
        d = 1/eigenValues[-K:]           #regularized inverse

    d = np.diag(d)   
    SqrtInvR= np.dot(eigenVectors[:,-K:], (np.dot( d, eigenVectors[:,-K:].T)))

    return SqrtInvR

def myCannoncorr(X,Y,Kx,Ky):
    Rxx,Ryy,Rxy,Ryx = nanRXY(X,Y)    

    # compute A
    Rxxnsq = regInv(Rxx,Kx,'sqrt'); # regularized Rxx^(-1/2)
    Ryyn = regInv(Ryy,Ky,'x')

    M = Rxxnsq.dot(Rxy).dot(Ryyn).dot(Ryx).dot(Rxxnsq)
    M = np.sum([M,M.T],axis=0) / 2   # fix nummerical precision asymmetric

    eigenValues,eigenVectors = LA.eigh(M)
    idx = np.argsort(eigenValues)
    #idx = idx[::-1]

    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]

    d = eigenVectors[:,-np.min((Kx,Ky)):]    
    A = Rxxnsq.dot(d)    # invert coordinate transformation


    # compute B
    Ryynsq=regInv(Ryy,Ky,'sqrt') # regularized Ryy^(-1/2)
    D=Ryynsq.dot(Ryx).dot(Rxxnsq).dot(d)
    B=Ryynsq.dot(D)


    U=A.T.dot(X)
    V=B.T.dot(Y)
    
    
    nVars= np.min((U.shape[0],V.shape[0]))
    rhos = np.zeros((nVars,1))
    pvals = np.zeros((nVars,1))

    for i in range(0,nVars):
        r,p = stats.pearsonr(U[i,:],V[i,:])
        rhos[i] = r
        pvals[i] = p
    
    
    return A,B,rhos,pvals,U,V,Rxx,Ryy

def get_component_topo_time(A,B,Ryy,temporal_aperature,nComp,new_sampling_rate):   
    b = B[:,0:nComp]/LA.norm(B[:,0:nComp])
    forwards = Ryy.dot(b).dot(LA.inv(b.T.dot(Ryy).dot(b)))  # B are the spatial filters applied to the EEG
    filters = A[:,0:nComp] # A are the filters applied to the kinematics    
    tvec=np.arange((-temporal_aperature/2+1),(temporal_aperature/2)+1)/new_sampling_rate
    
    return forwards,filters, tvec


#the temporal aperture -- how long to filter the kinematic signals (i.e., 1 second)
temporal_aperature = 400


Kx = 10    
Ky = 5
nComp=5

#### name
save_name = 'cca_yannis_Greaterthan_'+str(trial_len)+'sec_removeFirst_'\
            +str(remove_first)+'concatALLsub_'+str(delay[0])\
            +'delay_'+str(temporal_aperature)+'temporalAperature'


In [ ]:
# data selection functions
def get_eeg(raw,mat,resample_freq):
    trialno = mat['experiment']['media'][0,0]['permute'][0][0][0] 
    events_ = mne.find_events(raw, stim_channel='Trigger')
    a = events_[np.where(events_[:,2] == 105)[0],0]
    b = events_[np.where(events_[:,2] == 106)[0],0]

    a = a - raw.first_samp
    b = b - raw.first_samp

    A = raw.get_data()
    B=[]
    for i in range(0,len(a)):
        #c = signal.decimate(A[0:59,a[i]:b[i]], 10) # decimate to 200 hz 1000/100 =10
        c = A[0:59,a[i]:b[i]]
        x = c.shape[1]/1000 # Number of seconds in signal X
        x = x*resample_freq     # Number of samples to downsample
        c = scipy.signal.resample(c, int(np.ceil(x)),axis=1)
    
        df = pd.DataFrame({'trialno':trialno[i],'eeg':[c]})
        B.append(df)
    A = pd.concat((B),axis=0)
    clear_output()
    return A

def get_EMA(mat,feat):    
    trialno = mat['experiment']['media'][0,0]['permute'][0][0][0]    
    response = np.stack(mat['experiment']['media'][0,0]['Cresponse'][0][0].flatten()) - \
                np.stack(mat['experiment']['media'][0,0]['Sresponse'][0][0].flatten())    
    RT = np.stack(mat['experiment']['media'][0,0]['responseT'][0][0].flatten())   
    filename = np.stack(mat['experiment']['media'][0][0]['filename'][0][0][0]).flatten()
    df1 = pd.DataFrame({'trialno': range(200)})
    df1['trialno'] = trialno
    df1['response'] = response
    df1['RT'] = RT

    a = feat.merge(df1,on='trialno')
    
    return a

def align_data(B):
    eeg=[]
    ema=[]
    eeg_ema=[]
    for i in range(0,B.shape[0]):
        a = np.stack(B.iloc[i]['eeg']).shape[1]
        b = np.stack(B.iloc[i]['TTCD']).shape[0]
        
        x = np.stack((B.iloc[i]['envelop'].flatten(),
                       B.iloc[i]['jawaopening'].flatten(),
                       B.iloc[i]['lipaparature'].flatten(),
                       B.iloc[i]['lipProtrusion'].flatten(),
                       B.iloc[i]['TBCD'].flatten(),
                       B.iloc[i]['TMCD'].flatten(),
                       B.iloc[i]['TTCD'].flatten()))
        X=[]
        Y=[]
        if(b>a):
            X = B.iloc[i]['eeg']
            Y = x[:,:a]
        elif(a>b):
            X = B.iloc[i]['eeg'][:,:b]
            Y = x
        else:
            X = B.iloc[i]['eeg']
            Y = x
            
        eeg.append(X)
        ema.append(Y)    
        eeg_ema.append(np.vstack((X,Y)))
    return eeg,ema,eeg_ema


def select_portion_applyDelay(data,remove_first,d,sfreq,apply_delay):
    
    rs = 0.5*sfreq
    dd = d*sfreq
    L = []
    trial_no = len(data)
    eeg=[]
    ema=[]
    for tr in range(0,trial_no):
        # remove first from the begining
        aa = data[tr][:,int((remove_first)*sfreq):]
        
        if(apply_delay):
            trial_len = aa.shape[1] - 1*sfreq

            start_i = round(dd)
            end_i = round(dd+trial_len)
            start_s = round(rs)
            end_s = round(rs+trial_len)

            E = aa[0:59,int(start_i):int(end_i)]
            S = aa[-7:,int(start_s):int(end_s)]
        else:
            E = aa[0:59,:]
            S = aa[-7:,:]
        
        eeg.append(E)
        ema.append(S)
        L.append(S.shape[1])
        
    return eeg,ema,np.asarray(L)

def prepare_CCA_dataformat(eeg,ema,temporal_aperature,L,features):
    
    aa = np.hstack(ema)
    aa = stats.zscore(aa, axis=1)
    if(features=='envelop'):
        aa = aa[0,:]
    else:
        aa = aa[1:,:]
    mark_bad_trial = []
    X = []
    
    for t in range(0,len(ema)):
        a = aa[:,0:L[t]]
        #print(a.shape)
        tmp =[]
        for i in range(0,a.shape[0]):
            x = toeplitz(a[i,:])
            y= np.tril(np.ones((x.shape[0], x.shape[0]), dtype=int))
            x = x*y
            tmp.append(x[:,0:temporal_aperature])
        tmp = np.hstack(tmp)
        
        if(tmp.shape[1]==aa.shape[0]*temporal_aperature):
            tmp = np.hstack((tmp,np.ones((tmp.shape[0],1))))
            X.append(tmp) 
        else:
            mark_bad_trial.append(t)

        aa = aa[:,a.shape[1]:]

    ema = np.vstack(X)
    
    mark_bad_trial = np.asarray(mark_bad_trial)
    eeg_ = []
    for e in range(0,len(eeg)):
        if not(np.isin(e,mark_bad_trial)):        
            eeg_.append(eeg[e])

    eeg = np.hstack(eeg_)
    
    return eeg,ema.T,X,eeg_,mark_bad_trial

In [ ]:
# (greater than) trial_len and concat all the subjects in one delay
EEG = []
EMA = []
frame=[]
for s in range(0,len(subject_name)):
    raw_fname = data_path + '/python/data/rawEEG/'+subject_name[s]+'_raw.fif'
    raw = mne.io.read_raw_fif(raw_fname,preload=True)

    a = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
    raw_fname = a +'/exp/data/matlab_exp_data/'+subject_name[s]+'.mat'
    mat = scipy.io.loadmat(raw_fname)
    trialno = mat['experiment']['media'][0,0]['permute'][0][0][0] 

    eeg = get_eeg(raw,mat,new_sampling_rate)
    ema = get_EMA(mat,feat)
    A = eeg.merge(ema,on='trialno')

    #take only the correct response
    A = A[A['response']==0]

    # select trial length
    B = A[A['Trial_len']>=trial_len]

    # align both data
    eeg,ema,eeg_ema = align_data(B)

    #select portion with delay if any
    eeg,ema,L = select_portion_applyDelay(eeg_ema,remove_first,delay[0]+0.5,
                                          new_sampling_rate,apply_delay)
    
    #prepare for CCA
    eeg,ema,_,_,_, = prepare_CCA_dataformat(eeg,ema,temporal_aperature,L,features)
    
    EEG.append(eeg)
    EMA.append(ema)
    frame.append(eeg.shape[1])

EEG = np.hstack(EEG)    
EMA = np.hstack(EMA) 

In [ ]:
# MI measure

a = gcmi.gcmi_cc(eeg,ema)
